In [88]:
import re
import csv
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt

def extract_scaling_factors(svg_content):
    # Parse SVG content
    root = ET.fromstring(svg_content)

    # Extract text elements
    text_elements = root.findall(".//{http://www.w3.org/2000/svg}text")

    # Lists to hold x-coordinates
    x_coords = []

    for text in text_elements:
        try:
            value = int(text.text)
        except ValueError:
            continue  # Skip non-integer text elements

        # Extract transform matrix
        transform = text.get('transform', '')
        if 'matrix(' not in transform:
            continue  # Skip text elements without a matrix transform

        # Split transform matrix into components
        matrix = [float(val) for val in transform.replace('matrix(', '').replace(')', '').split()]
        
        # Append x-coordinate to the list
        x_coords.append(matrix[4])

    # Calculate median of x-coordinates
    median_x = sorted(x_coords)[len(x_coords) // 2]

    x_labels = {}
    y_labels = {}

    for text in text_elements:
        try:
            value = int(text.text)
            transform = text.get('transform', '')
            matrix = [float(val) for val in transform.replace('matrix(', '').replace(')', '').split()]
            
            # Classify based on median_x
            if matrix[4] >= median_x:
                x_labels[value] = matrix[4]
            else:
                y_labels[value] = matrix[5]
        except:
            continue

    print("x_labels:", x_labels)
    print("y_labels:", y_labels)

    # Ensure there are at least two labels
    if len(x_labels) < 2 or len(y_labels) < 2:
        raise ValueError("Not enough labels to calculate scaling factors")

    # Calculate scaling factors
    x_scale = (list(x_labels.keys())[2] - list(x_labels.keys())[1]) / (list(x_labels.values())[2] - list(x_labels.values())[1])
    y_scale = (list(y_labels.keys())[2] - list(y_labels.keys())[1]) / (list(y_labels.values())[2] - list(y_labels.values())[1])

    origin_x, origin_y = float('inf'), float(0)  # set initial values to infinity

    # Find all paths with the specified stroke color (#262626)
    gray_pattern = r'<path[^>]*?stroke="#262626"[^>]*?>'
    gray_matches = re.findall(gray_pattern, svg_content)

    # Iterate through matched gray paths and extract coordinates
    for gray_path in gray_matches:
        d_pattern = r'd="([^"]+)"'
        d_match = re.search(d_pattern, gray_path)
        if d_match:
            path_data = d_match.group(1)
            parts = re.split(r'(M|L)', path_data)[1:]
            for i in range(0, len(parts), 2):
                command = parts[i]
                coords = re.findall(r'([\d.-]+)[ ,]([\d.-]+)', parts[i+1])
                for x, y in coords:
                    origin_x = min(origin_x, float(x))
                    origin_y = max(origin_y, float(y))


    return x_scale, y_scale, origin_x, origin_y  

def extract_svg_data(filename, colors, head, suffix = None):
    # Read SVG from file
    with open(filename + '.svg', 'r') as file:
        svg_content = file.read()

    # Find scaling factors
    x_scale, y_scale, origin_x, origin_y = extract_scaling_factors(svg_content)
    print("X Scale:", x_scale)
    print("Y Scale:", y_scale)
    print("Origin X:", origin_x)
    print("Origin Y:", origin_y)


    all_coordinates = []

    for color in colors:
        # Find all paths with the current stroke color
        pattern = rf'<path[^>]*?stroke="{color}"[^>]*?>'
        path_matches = re.findall(pattern, svg_content)
        
        if not path_matches:
            print(f"Paths with color {color} not found.")
            continue
        
        for idx, path_string in enumerate(path_matches):
            # From each path string, extract the d attribute
            d_pattern = r'd="([^"]+)"'
            d_match = re.search(d_pattern, path_string)
            
            if not d_match:
                print(f"Path data not found in the matched path #{idx + 1}.")
                continue
            
            path_data = d_match.group(1)
            print(f"Extracted path data for plot #{idx + 1}: {path_data}")  # Debug print
            
            # Extract coordinates using regex
            coordinates = []
            parts = re.split(r'(M|L)', path_data)[1:]
            for i in range(0, len(parts), 2):
                command = parts[i]
                coords = re.findall(r'([\d.-]+)[ ,]([\d.-]+)', parts[i+1])
                coordinates.extend((float(x), float(y)) for x, y in coords)
            
            scaled_coordinates = []
            scaled_coordinates.extend(((x-origin_x)*x_scale,(y-origin_y)*y_scale) for x, y in coordinates)
            print(f"Extracted coordinates for plot #{idx + 1}: {scaled_coordinates}")  # Debug print
            
            # Append all coordinates to the main list
            all_coordinates.append(scaled_coordinates)

    # Provide ability to add suffix to filename
    if suffix != None: filename += suffix
    
    # Save all extracted coordinates to a single CSV file
    with open(filename + '.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        headers = []
        for i in range(len(all_coordinates)):
            headers.extend([head[0]+f"_{i}", head[1]+f"_{i}"])
        csvwriter.writerow(headers)
        for row in zip(*all_coordinates):
            csvwriter.writerow([coord for coords in row for coord in coords])

mode = "torque"

if mode == "force":
    extract_svg_data('simple-printed-force',['#4DAF4A',"#AFDBAE"],["Displacment","Force"])
    extract_svg_data('simple-steel-force',['#4DAF4A',"#AFDBAE"],["Displacment","Force"])
    extract_svg_data('truss-steel-force',['#4DAF4A',"#AFDBAE"],["Displacment","Force"])

elif mode == "torque":
    extract_svg_data('simple-printed-torque',['#377EB8',"#A5C5DF"],["Rotation","Torque"], suffix = "_azimuth")
    extract_svg_data('simple-printed-torque',['#E41A1C',"#F39899"],["Rotation","Torque"], suffix = "_pole")

elif mode == "flex":
    extract_svg_data('flex-shaft',['#984EA3',"#D1AFD6"],["Strain","Stifness"])



IndexError: list index out of range

In [98]:
import re
import csv
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt

def extract_scaling_factors(svg_content):

    origin_x, origin_y = float('inf'), float(0)  # set initial values to infinity

    # Find all paths with the specified stroke color (#262626)
    gray_pattern = r'<path[^>]*?stroke="#262626"[^>]*?>'
    gray_matches = re.findall(gray_pattern, svg_content)

    # Iterate through matched gray paths and extract coordinates
    for gray_path in gray_matches:
        d_pattern = r'd="([^"]+)"'
        d_match = re.search(d_pattern, gray_path)
        if d_match:
            path_data = d_match.group(1)
            parts = re.split(r'(M|L)', path_data)[1:]
            for i in range(0, len(parts), 2):
                command = parts[i]
                coords = re.findall(r'([\d.-]+)[ ,]([\d.-]+)', parts[i+1])
                for x, y in coords:
                    origin_x = min(origin_x, float(x))
                    origin_y = max(origin_y, float(y))


    return origin_x, origin_y

def extract_svg_data(filename, colors, head, suffix = None, x_scale = None, y_scale = None):
    # Read SVG from file
    with open(filename + '.svg', 'r') as file:
        svg_content = file.read()

    # Find scaling factors
    origin_x, origin_y = extract_scaling_factors(svg_content)
    print("X Scale:", x_scale)
    print("Y Scale:", y_scale)
    print("Origin X:", origin_x)
    print("Origin Y:", origin_y)


    all_coordinates = []

    for color in colors:
        # Find all paths with the current stroke color
        pattern = rf'<path[^>]*?stroke="{color}"[^>]*?>'
        path_matches = re.findall(pattern, svg_content)
        
        if not path_matches:
            print(f"Paths with color {color} not found.")
            continue
        
        for idx, path_string in enumerate(path_matches):
            # From each path string, extract the d attribute
            d_pattern = r'd="([^"]+)"'
            d_match = re.search(d_pattern, path_string)
            
            if not d_match:
                print(f"Path data not found in the matched path #{idx + 1}.")
                continue
            
            path_data = d_match.group(1)
            print(f"Extracted path data for plot #{idx + 1}: {path_data}")  # Debug print
            
            # Extract coordinates using regex
            coordinates = []
            parts = re.split(r'(M|L)', path_data)[1:]
            for i in range(0, len(parts), 2):
                command = parts[i]
                coords = re.findall(r'([\d.-]+)[ ,]([\d.-]+)', parts[i+1])
                coordinates.extend((float(x), float(y)) for x, y in coords)
            
            scaled_coordinates = []
            scaled_coordinates.extend(((x-origin_x)*x_scale,(y-origin_y)*y_scale) for x, y in coordinates)
            print(f"Extracted coordinates for plot #{idx + 1}: {scaled_coordinates}")  # Debug print
            
            # Append all coordinates to the main list
            if len(scaled_coordinates) > 2:
                all_coordinates.append(scaled_coordinates)

    # Provide ability to add suffix to filename
    if suffix != None: filename += suffix
    
    # Save all extracted coordinates to a single CSV file
    with open(filename + '.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        headers = []
        for i in range(len(all_coordinates)):
            headers.extend([head[0]+f"_{i}", head[1]+f"_{i}"])
        csvwriter.writerow(headers)
        for row in zip(*all_coordinates):
            csvwriter.writerow([coord for coords in row for coord in coords])

x_s = 0.1/(114.38-82.49)
y_s = 20/(14.8-54.77)
extract_svg_data('simple-printed-torque',['#377EB8',"#A5C5DF"],["Rotation","Torque"], suffix = "_azimuth",x_scale=x_s,y_scale=y_s)
extract_svg_data('simple-printed-torque',['#E41A1C',"#F39899"],["Rotation","Torque"], suffix = "_pole",x_scale=x_s,y_scale=y_s)

x_s = 0.1/(114.38-82.49)
y_s = 20/(8.63-49.94)
extract_svg_data('simple-steel-torque',['#377EB8',"#A5C5DF"],["Rotation","Torque"], suffix = "_azimuth",x_scale=x_s,y_scale=y_s)
extract_svg_data('simple-steel-torque',['#E41A1C',"#F39899"],["Rotation","Torque"], suffix = "_pole",x_scale=x_s,y_scale=y_s)

x_s = 0.1/(114.56-83.05)
y_s = 50/(17.9-56.52)
extract_svg_data('truss-steel-torque',['#377EB8',"#A5C5DF"],["Rotation","Torque"], suffix = "_azimuth",x_scale=x_s,y_scale=y_s)
extract_svg_data('truss-steel-torque',['#E41A1C',"#F39899"],["Rotation","Torque"], suffix = "_pole",x_scale=x_s,y_scale=y_s)



X Scale: 0.0031357792411414237
Y Scale: -0.5003752814610959
Origin X: 18.6875
Origin Y: 97.4625
Extracted path data for plot #1: M18.6875 94.7502 19.2359 94.8054 19.7843 94.5093 20.3327 94.1384 20.8813 93.6557 21.4297 93.3032 21.9781 92.915 22.5265 92.5524 23.0749 92.1623 23.6233 91.891 24.1717 91.4914 24.7203 91.1243 25.2687 90.8007 25.8171 90.4392 26.3655 90.1509 26.9139 89.7917 27.4623 89.4461 28.0107 89.1276 28.5592 88.7732 29.1077 88.4579 29.6561 88.1238 30.2045 87.8813 30.7529 87.4727 31.3013 87.1783 31.8497 86.8865 32.3981 86.5612 32.9466 86.2616 33.495 85.9793 34.0434 85.6973 34.5919 85.3599 35.1403 85.0641 35.6887 84.7783 36.2371 84.471 36.7856 84.1511 37.334 83.9136 37.8824 83.6175 38.4308 83.3563 38.9792 83.0642 39.5277 82.8108 40.0761 82.4843 40.6246 82.2496 41.173 81.9526 41.7214 81.6951 42.2698 81.3993 42.8182 81.115 43.3666 80.8588 43.9151 80.6295 44.4635 80.3693 45.012 80.0673 45.5604 79.8748 46.1088 79.5297 46.6572 79.2699 47.2056 79.0626 47.754 78.8086 48.3024 78.5829